In [1]:
import torch
import torchvision
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split



In [3]:
train =pd.read_csv("data.zip/fashion-mnist_train.csv")
test = pd.read_csv("data.zip/fashion-mnist_test.csv")

FileNotFoundError: [Errno 2] File data.zip/fashion-mnist_train.csv does not exist: 'data.zip/fashion-mnist_train.csv'

In [4]:
train.head()

NameError: name 'train' is not defined

In [9]:
class MyDataset():
    '''
    Build your own dataset
    '''
    def __init__(self, data, transform = None):
        self.fashion_mnist = list(data.values)
        self.transform = transform
        label, img = [],[]
        for one_line in self.fashion_mnist:
            label.append(one_line[0])
            img.append(one_line[1:])
        self.label = np.asarray(label)
        self.img = np.asarray(img).reshape(-1, 28, 28,1).astype('float32')

    def __getitem__(self, item):
        label, img = self.label[item], self.img[item]
        if self.transform is not None:
            img = self.transform(img)

        return  img,label

    def __len__(self):
        return len(self.label)

In [10]:
My_transform = transforms.Compose([
    transforms.ToTensor(),
])

Train_data = MyDataset(train, transform=My_transform)
Test_data = MyDataset(test, transform=My_transform)

Train_dataloader = DataLoader(dataset=Train_data,
                              batch_size = 128,
                              shuffle=True
                              )
Test_dataloader = DataLoader(dataset=Test_data,
                              batch_size = 128,
                              shuffle=False
                              )

In [11]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 10)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

net = Net()
print(net)

Net(
  (fc1): Linear(in_features=784, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=16, bias=True)
  (fc4): Linear(in_features=16, out_features=10, bias=True)
)


In [12]:
import torch.optim as optim

loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [14]:
for epoch in range(5): 
    for data in Train_dataloader:  
        X,y = data  
        net.zero_grad()  
        output = net(X.view(-1,784))  
        loss = F.nll_loss(output, y)  
        loss.backward()  
        optimizer.step()  
    print(loss)  

tensor(0.3880, grad_fn=<NllLossBackward>)
tensor(0.4233, grad_fn=<NllLossBackward>)
tensor(0.3451, grad_fn=<NllLossBackward>)
tensor(0.3315, grad_fn=<NllLossBackward>)
tensor(0.3193, grad_fn=<NllLossBackward>)


In [15]:
for epoch in range(5): 
    for data in Test_dataloader:  
        X,y = data  
        net.zero_grad()  
        output = net(X.view(-1,784))  
        loss = F.nll_loss(output, y)  
        loss.backward()  
        optimizer.step()  
    print(loss)  

tensor(0.3012, grad_fn=<NllLossBackward>)
tensor(0.1457, grad_fn=<NllLossBackward>)
tensor(0.0960, grad_fn=<NllLossBackward>)
tensor(0.0552, grad_fn=<NllLossBackward>)
tensor(0.0517, grad_fn=<NllLossBackward>)
